In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from
df = pd.read_csv("C:/Users/pedro/Documents/GitHub/IronHackLabs/Week2/Data_Marketing_Customer_Analysis_Round3.csv")
df = pd.DataFrame(df)
df

In [ ]:
#Separing train and test samples
TT_RATIO = 0.3
RAND_STATE = 40
x = df.drop('total_claim_amount', axis=1)
y = df.total_claim_amount
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TT_RATIO, random_state=RAND_STATE)
x_train = pd.DataFrame(x_train)
x_test = pd.DataFrame(x_test)
numericalX = x.select_dtypes(np.number)
categoricalx = x.select_dtypes(object)

In [ ]:
#Normalize the data
pt = PowerTransformer()
ct = ColumnTransformer([("pt", pt, list(numericalX.columns))],
                        remainder='drop',verbose_feature_names_out=True,verbose=True).fit(x_train)
x_train_ct = pd.DataFrame(ct.transform(x_train),columns=ct.get_feature_names_out())
x_test_ct = pd.DataFrame(ct.transform(x_test),columns=ct.get_feature_names_out())

In [ ]:
#Make the normalized data a DataFrame
x_train = pd.DataFrame(x_train_ct)

In [ ]:
#Train the model using Ordinary least squares
x_train_const_ct = sm.add_constant(x_train_ct.to_numpy()) # adding a constant

model = sm.OLS(y_train, x_train_const_ct).fit()
predictions_train = model.predict(x_train_const_ct)

x_test_const_ct = sm.add_constant(x_test_ct) # adding a constant
predictions_test = model.predict(x_test_const_ct)
print_model = model.summary()
print(print_model)

In [ ]:
#Train the model using linear regression.
model2=LinearRegression()    # model
model2.fit(x_train_ct, y_train)   # model train

In [ ]:
#Check the coeficiente. Part have a negative correlation, part positive.
model2.coef_

In [ ]:
#Checking the constant value.
model2.intercept_

In [ ]:
#Make the predictions.
y_pred = pd.DataFrame(model2.predict(x_test_ct),columns = ['total_claim_amount'] )      # model prediction
y_pred_train =  pd.DataFrame(model2.predict(x_train_ct),columns = ['total_claim_amount'])

In [ ]:
#Model Evaluation: MSE, MAE, RMSE. The model is not that great.
print(mse(y_test,y_pred))
mse = mse(y_test, y_pred)
print(mae(y_test,y_pred))
np.sqrt(mse)

In [17]:
R2=r2_score(y_test,y_pred) #the R2 is too low, showing that the model is not good for prediction.
R2
Adj_R2= 1 - (1-R2)*(len(y_test)-1)/(len(y_test)-x_test.shape[1]-1)
Adj_R2

0.28446313398568024

In [21]:
#Feature Importance to check what columns are the most important.
feature_importances = pd.DataFrame(data= {'Attribute': x_train.columns, 'Importance': abs(model2.coef_.reshape(len(x_train.columns),))})
feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

In [23]:
#Monthly_premium_auto is the most relevant feature for the prediction.
feature_importances

,Attribute,Importance
2,pt__monthly_premium_auto,163.794492
1,pt__income,35.917429
5,pt__number_of_open_complaints,5.554372
0,pt__customer_lifetime_value,4.050760
6,pt__number_of_policies,2.362977
3,pt__months_since_last_claim,1.201657
4,pt__months_since_policy_inception,0.358445


In [24]:
#Separating nominals and ordinals
c1 = categoricalx[['region','response', 'gender', 'marital_status', 'employment_status', 'sales_channel', 'vehicle_class', 'policy_type']].copy()
c2 = categoricalx[['coverage', 'education', 'month', 'policy', 'renew_offer_type', 'vehicle_size']].copy()
c2

,coverage,education,month,policy,renew_offer_type,vehicle_size
0,basic,college,feb,corporate l3,offer3,medsize
1,basic,college,jan,personal l3,offer4,medsize
2,basic,bachelor,feb,personal l3,offer3,medsize
3,extended,college,jan,corporate l3,offer2,medsize
4,premium,bachelor,jan,personal l2,offer1,medsize
...,...,...,...,...,...,...
10684,premium,bachelor,jan,personal l1,offer3,medsize
10685,basic,college,jan,personal l3,offer2,medsize
10686,extended,bachelor,feb,corporate l3,offer1,medsize
10687,premium,college,feb,personal l1,offer1,medsize


In [25]:
c1 = pd.get_dummies(c1) #dummies

#create a hierarchy for the values
mapper = {'basic': 0, 'extended': 1, 'premium': 2, 'high school or below': 0, 'bachelor': 1, 'college': 2, 'master': 3, 'doctor': 4, 'jan': 0, 'feb': 1, 'small': 0, 'medsize': 1, 'large': 2, 'corporate l3': 6, 'personal l3': 3, 'personal l2': 2, 'corporate l2': 5,'personal l1': 1, 'special l1': 7, 'corporate l1': 4, 'special l3': 9,
'special l2': 8}
#add new columns
c2['coverage'] = c2['coverage'].replace(mapper)
c2['education'] = c2['education'].replace(mapper)
c2['month'] = c2['month'].replace(mapper)
c2['policy'] = c2['policy'].replace(mapper)
c2['renew_offer_type'] = c2['renew_offer_type'].replace(mapper)
c2['vehicle_size'] = c2['vehicle_size'].replace(mapper)
c2

,coverage,education,month,policy,renew_offer_type,vehicle_size
0,0,2,1,6,offer3,1
1,0,2,0,3,offer4,1
2,0,1,1,3,offer3,1
3,1,2,0,6,offer2,1
4,2,1,0,2,offer1,1
...,...,...,...,...,...,...
10684,2,1,0,1,offer3,1
10685,0,2,0,3,offer2,1
10686,1,1,1,6,offer1,1
10687,2,2,1,1,offer1,1


In [26]:
#Rerun the model after adding the hot encoded categorical variables
TT_RATIO = 0.3
RAND_STATE = 40
z_train, z_test, = train_test_split(c1, test_size=TT_RATIO, random_state=RAND_STATE)
z_train = pd.DataFrame(z_train)
z_test = pd.DataFrame(z_test)
z_train

,region_central,region_east,region_north west,region_west region,response_no,response_yes,gender_f,gender_m,marital_status_divorced,marital_status_married,...,sales_channel_web,vehicle_class_four-door car,vehicle_class_luxury car,vehicle_class_luxury suv,vehicle_class_sports car,vehicle_class_suv,vehicle_class_two-door car,policy_type_corporate auto,policy_type_personal auto,policy_type_special auto
7388,False,False,True,False,True,False,False,True,False,True,...,False,False,False,False,False,False,True,False,True,False
9559,False,False,True,False,True,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
3163,False,False,False,True,True,False,False,True,False,True,...,True,False,False,True,False,False,False,False,True,False
338,False,False,True,False,True,False,True,False,False,False,...,False,False,False,False,False,True,False,True,False,False
3501,True,False,False,False,True,False,True,False,False,True,...,False,False,False,False,False,False,True,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7839,False,True,False,False,True,False,False,True,True,False,...,True,False,False,False,False,True,False,False,True,False
3603,False,False,True,False,True,False,True,False,True,False,...,False,True,False,False,False,False,False,True,False,False
5959,False,False,False,True,True,False,True,False,False,True,...,False,True,False,False,False,False,False,False,True,False
5426,False,False,False,True,True,False,True,False,False,True,...,False,True,False,False,False,False,False,False,True,False
